In [1]:
# install boto3 for AWS connection
dbutils.library.installPyPI("boto3", version="1.9.157")
dbutils.library.restartPython()

In [2]:
# install these Maven packages in the cluster
# JohnSnowLabs:spark-nlp:2.3.4

In [3]:
# import AWS credentials
# import config.py ##for local
%run "/dbfs/FileStore/tables/config" ##for databricks

In [4]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [5]:
# get or create Spark session
app_name = "spark-airbnb-sentiment"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [6]:
import boto3

secret_name = my_secret_name
region_name = my_region_name
access_key  = my_access_key
secret_key  = my_secret_key

session      = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client       = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)

In [7]:
import json
def get_connection(secret_value):
    return json.loads(secret_value['SecretString'])

In [8]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort     = connection['port']
jdbcDatabase = "postgres"
dialect      = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user"     : jdbcUsername,
  "password" : jdbcPassword,
  "driver"   : "org.postgresql.Driver" 
}

In [9]:
# Read from reviews_full table

table = "reviews_full"

df = spark.read.jdbc(url=jdbcUrl, table=table, properties=connectionProperties)
df.printSchema()

root
-- listing_id: integer (nullable = true)
-- id: integer (nullable = true)
-- date: date (nullable = true)
-- reviewer_id: integer (nullable = true)
-- reviewer_name: string (nullable = true)
-- comments: string (nullable = true)

In [10]:
display(df.select(df.columns))

listing_id,id,date,reviewer_id,reviewer_name,comments
3862246,54663524,2015-11-22,985696,Lydia,"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!"
3862246,56898680,2015-12-19,17020551,Leslie,"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described."
3862246,58253716,2016-01-01,21499045,Jackson,Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.
3862246,58445874,2016-01-02,8313146,Jaime,"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic."
3862246,58584255,2016-01-03,2806995,Rachel,"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return."
3862246,59372905,2016-01-10,48798728,Charlotte,"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! So - if you're going to Austin, stay here!"
3862246,60522554,2016-01-24,26792421,Rubie,"Denise was SO nice and welcoming! :) She stayed in contact with us as we traveled and made it easy to check in with her and get settled. She went out of her way to make our stay there enjoyable in many little ways. She provided some essentials that other airBNB's don't like spare toothbrushes, lotion, etc. she gave a great recommendation for lunch too! The room is very clean and comfortable and you really can have as much privacy as you want with your own separate entrance, but Denise was also super friendly and willing to help in any way. Overall we had a great trip!"
3862246,61017340,2016-01-30,30715591,Christi,The host canceled this reservation the day before arrival. This is an automated posting.
3862246,66008610,2016-03-18,15770643,Val,Peter was helpful in arranging an earlier check-in time so I could drop my bags off before heading off to SXSW. He showed me around and the backyard was awesome - I was bummed to not be able to spend more time there and sit out in the yard. The neighborhood is quite but a short ride away from downtown. The room is perfect if you're going to be out and about exploring. Thanks for everything.
3862246,66933457,2016-03-25,62149775,Scott,Peter made me feel more than welcome in his cute house in a very convenient side of Austin. I loved the proximity to the coffee shop as well! Peter treated me like a long-lost cousin in that I never felt like a stranger being tolerated in someone's home! He always answered questions rapidly and even met me at

In [11]:
from pyspark.ml import Pipeline
import sparknlp
sparknlp.start()
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline("analyze_sentiment", lang="en")

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [12]:
comments1 = first_row.select("comments").collect()

In [13]:
comments1 = comments1[0].comments

In [14]:
result = pipeline.annotate(comments1)

In [15]:
print(result['sentiment'])

['negative', 'positive', 'positive', 'positive', 'negative']

In [16]:
print(result['checked'])

['Denise', 'was', 'super', 'friendly', 'and', 'her', 'home', 'was', 'impeccably', 'clean', 'and', 'beyond', 'charming', '.', 'Her', 'two', 'little', 'dogs', 'were', 'quiet', 'friendly', 'and', 'the', 'separate', 'entrance', 'was', 'very', 'nice', '.', 'Have', 'never', 'seen', 'such', 'a', 'well', 'stocked', 'bathroom', ',', 'an', 'keurig', 'right', 'in', 'the', 'bedroom', ',', 'and', 'excellent', 'water', 'pressure', '!', 'A', 'great', 'representation', 'and', 'slice', 'of', 'Austin', '.', 'Would', 'highly', 'recommend', 'staying', 'with', 'her', '!']

In [17]:
thousand_row = df.select(df.columns).limit(1000)

In [18]:
annotations_df = pipeline.transform(thousand_row.withColumnRenamed("comments", "text"))
# display(annotations_df)

In [19]:
# return the sentiment with the highest confidence value
@F.udf()
def highestConfidence(sentClassifications):
  idx = np.argmax([x["metadata"]["confidence"] for x in sentClassifications])
  return sentClassifications[idx]["result"]

In [20]:
# return sentiment of the longest sentence
@F.udf()
def longestSentiment(sentClassifications):
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSent = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSent = sum([x[1] for x in sentLengths if x[0] == "negative"])
  return "positive" if posSent>negSent else "negative"

In [21]:
# return the overall average sentiment
@F.udf()
def averageSentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSent = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSent = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  return "positive" if posSent>=negSent else "negative"

In [22]:
# null comments exist so must account for them
print(df.filter("comments is null").count())

# return the overall average sentiment after normalizing by sentence length
@F.udf()
def classifySentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSentLengths = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSentLengths = sum([x[1] for x in sentLengths if x[0] == "negative"])
  
  # set to NaN if we have null comments
  if posSentLengths == 0 and negSentLengths == 0:
    return np.nan
  
  if abs(posSentLengths - negSentLengths)/(posSentLengths + negSentLengths) > 0.2:
    return "positive" if posSentLengths>negSentLengths else "negative"
  
  else:
    return "positive" if posSentConf>negSentConf else "negative"

In [23]:
display(annotations_df.select("text", "sentiment", averageSentiment("sentiment")))

text,sentiment,averageSentiment(sentiment)
"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!","List(List(sentiment, 0, 79, negative, Map(confidence -> 0.4855), List()), List(sentiment, 81, 160, positive, Map(confidence -> 0.5427), List()), List(sentiment, 162, 268, positive, Map(confidence -> 0.5373), List()), List(sentiment, 270, 312, positive, Map(confidence -> 0.4801), List()), List(sentiment, 315, 354, negative, Map(confidence -> 0.4508), List()))",positive
"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described.","List(List(sentiment, 0, 43, negative, Map(confidence -> 0.5164), List()), List(sentiment, 45, 101, positive, Map(confidence -> 0.5029), List()))",negative
Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.,"List(List(sentiment, 0, 71, positive, Map(confidence -> 0.5105), List()), List(sentiment, 73, 118, positive, Map(confidence -> 0.4875), List()), List(sentiment, 120, 156, negative, Map(confidence -> 0.4736), List()))",positive
"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic.","List(List(sentiment, 0, 37, positive, Map(confidence -> 0.5918), List()), List(sentiment, 39, 66, positive, Map(confidence -> 0.6151), List()), List(sentiment, 68, 134, negative, Map(confidence -> 0.4904), List()), List(sentiment, 136, 193, negative, Map(confidence -> 0.5118), List()), List(sentiment, 196, 220, positive, Map(confidence -> 0.8270), List()), List(sentiment, 222, 257, positive, Map(confidence -> 0.5258), List()), List(sentiment, 259, 290, negative, Map(confidence -> 0.4657), List()), List(sentiment, 292, 333, negative, Map(confidence -> 0.4841), List()), List(sentiment, 335, 367, negative, Map(confidence -> 0.5114), List()))",positive
"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return.","List(List(sentiment, 0, 56, negative, Map(confidence -> 0.4874), List()), List(sentiment, 58, 183, positive, Map(confidence -> 0.5033), List()), List(sentiment, 185, 288, negative, Map(confidence -> 0.5061), List()), List(sentiment, 290, 364, positive, Map(confidence -> 0.5433), List()), List(sentiment, 366, 388, positive, Map(confidence -> 0.4955), List()), List(sentiment, 390, 466, positive, Map(confidence -> 0.5481), List()))",positive
"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! S

In [24]:
display(annotations_df.select("text", "sentiment", highestConfidence("sentiment")))

text,sentiment,highestConfidence(sentiment)
"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!","List(List(sentiment, 0, 79, negative, Map(confidence -> 0.4855), List()), List(sentiment, 81, 160, positive, Map(confidence -> 0.5427), List()), List(sentiment, 162, 268, positive, Map(confidence -> 0.5373), List()), List(sentiment, 270, 312, positive, Map(confidence -> 0.4801), List()), List(sentiment, 315, 354, negative, Map(confidence -> 0.4508), List()))",positive
"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described.","List(List(sentiment, 0, 43, negative, Map(confidence -> 0.5164), List()), List(sentiment, 45, 101, positive, Map(confidence -> 0.5029), List()))",negative
Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.,"List(List(sentiment, 0, 71, positive, Map(confidence -> 0.5105), List()), List(sentiment, 73, 118, positive, Map(confidence -> 0.4875), List()), List(sentiment, 120, 156, negative, Map(confidence -> 0.4736), List()))",positive
"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic.","List(List(sentiment, 0, 37, positive, Map(confidence -> 0.5918), List()), List(sentiment, 39, 66, positive, Map(confidence -> 0.6151), List()), List(sentiment, 68, 134, negative, Map(confidence -> 0.4904), List()), List(sentiment, 136, 193, negative, Map(confidence -> 0.5118), List()), List(sentiment, 196, 220, positive, Map(confidence -> 0.8270), List()), List(sentiment, 222, 257, positive, Map(confidence -> 0.5258), List()), List(sentiment, 259, 290, negative, Map(confidence -> 0.4657), List()), List(sentiment, 292, 333, negative, Map(confidence -> 0.4841), List()), List(sentiment, 335, 367, negative, Map(confidence -> 0.5114), List()))",positive
"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return.","List(List(sentiment, 0, 56, negative, Map(confidence -> 0.4874), List()), List(sentiment, 58, 183, positive, Map(confidence -> 0.5033), List()), List(sentiment, 185, 288, negative, Map(confidence -> 0.5061), List()), List(sentiment, 290, 364, positive, Map(confidence -> 0.5433), List()), List(sentiment, 366, 388, positive, Map(confidence -> 0.4955), List()), List(sentiment, 390, 466, positive, Map(confidence -> 0.5481), List()))",positive
"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! 

In [25]:
display(annotations_df.select("text", "sentiment", longestSentiment("sentiment")))

text,sentiment,longestSentiment(sentiment)
"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!","List(List(sentiment, 0, 79, negative, Map(confidence -> 0.4855), List()), List(sentiment, 81, 160, positive, Map(confidence -> 0.5427), List()), List(sentiment, 162, 268, positive, Map(confidence -> 0.5373), List()), List(sentiment, 270, 312, positive, Map(confidence -> 0.4801), List()), List(sentiment, 315, 354, negative, Map(confidence -> 0.4508), List()))",positive
"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described.","List(List(sentiment, 0, 43, negative, Map(confidence -> 0.5164), List()), List(sentiment, 45, 101, positive, Map(confidence -> 0.5029), List()))",positive
Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.,"List(List(sentiment, 0, 71, positive, Map(confidence -> 0.5105), List()), List(sentiment, 73, 118, positive, Map(confidence -> 0.4875), List()), List(sentiment, 120, 156, negative, Map(confidence -> 0.4736), List()))",positive
"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic.","List(List(sentiment, 0, 37, positive, Map(confidence -> 0.5918), List()), List(sentiment, 39, 66, positive, Map(confidence -> 0.6151), List()), List(sentiment, 68, 134, negative, Map(confidence -> 0.4904), List()), List(sentiment, 136, 193, negative, Map(confidence -> 0.5118), List()), List(sentiment, 196, 220, positive, Map(confidence -> 0.8270), List()), List(sentiment, 222, 257, positive, Map(confidence -> 0.5258), List()), List(sentiment, 259, 290, negative, Map(confidence -> 0.4657), List()), List(sentiment, 292, 333, negative, Map(confidence -> 0.4841), List()), List(sentiment, 335, 367, negative, Map(confidence -> 0.5114), List()))",negative
"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return.","List(List(sentiment, 0, 56, negative, Map(confidence -> 0.4874), List()), List(sentiment, 58, 183, positive, Map(confidence -> 0.5033), List()), List(sentiment, 185, 288, negative, Map(confidence -> 0.5061), List()), List(sentiment, 290, 364, positive, Map(confidence -> 0.5433), List()), List(sentiment, 366, 388, positive, Map(confidence -> 0.4955), List()), List(sentiment, 390, 466, positive, Map(confidence -> 0.5481), List()))",positive
"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! S

In [26]:
display(annotations_df.select("text", "sentiment", classifySentiment("sentiment")))

text,sentiment,classifySentiment(sentiment)
"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!","List(List(sentiment, 0, 79, negative, Map(confidence -> 0.4855), List()), List(sentiment, 81, 160, positive, Map(confidence -> 0.5427), List()), List(sentiment, 162, 268, positive, Map(confidence -> 0.5373), List()), List(sentiment, 270, 312, positive, Map(confidence -> 0.4801), List()), List(sentiment, 315, 354, negative, Map(confidence -> 0.4508), List()))",positive
"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described.","List(List(sentiment, 0, 43, negative, Map(confidence -> 0.5164), List()), List(sentiment, 45, 101, positive, Map(confidence -> 0.5029), List()))",negative
Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.,"List(List(sentiment, 0, 71, positive, Map(confidence -> 0.5105), List()), List(sentiment, 73, 118, positive, Map(confidence -> 0.4875), List()), List(sentiment, 120, 156, negative, Map(confidence -> 0.4736), List()))",positive
"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic.","List(List(sentiment, 0, 37, positive, Map(confidence -> 0.5918), List()), List(sentiment, 39, 66, positive, Map(confidence -> 0.6151), List()), List(sentiment, 68, 134, negative, Map(confidence -> 0.4904), List()), List(sentiment, 136, 193, negative, Map(confidence -> 0.5118), List()), List(sentiment, 196, 220, positive, Map(confidence -> 0.8270), List()), List(sentiment, 222, 257, positive, Map(confidence -> 0.5258), List()), List(sentiment, 259, 290, negative, Map(confidence -> 0.4657), List()), List(sentiment, 292, 333, negative, Map(confidence -> 0.4841), List()), List(sentiment, 335, 367, negative, Map(confidence -> 0.5114), List()))",negative
"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return.","List(List(sentiment, 0, 56, negative, Map(confidence -> 0.4874), List()), List(sentiment, 58, 183, positive, Map(confidence -> 0.5033), List()), List(sentiment, 185, 288, negative, Map(confidence -> 0.5061), List()), List(sentiment, 290, 364, positive, Map(confidence -> 0.5433), List()), List(sentiment, 366, 388, positive, Map(confidence -> 0.4955), List()), List(sentiment, 390, 466, positive, Map(confidence -> 0.5481), List()))",positive
"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! 